In [91]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [356]:
def successor(X, s, r, m, T):
    h = T/m
    z = np.random.normal()
    return X * np.exp(h * (r - 0.05 - (s**2)/2) + s * np.sqrt(h) * z)

def black_scholes(X0, s, r, m, T):
    X = [X0]
    for i in range(m):
        X.append(successor(X[-1], s, r, m, T))
    return X

def put_func(a, K):
    """
    Payoff function for a Bermudan put option
    """
    return np.maximum(K - a, 0)

def call_func(a, K):
    """
    Payoff function for a Bermudan call option
    """
    return np.maximum(a - K, 0)

def high_estimator(payoff_func, current, nextt, K, r, h):
    b = len(nextt)
    discount_factor = np.exp(-r*h)
    return max(payoff_func(current, K), discount_factor * np.sum(nextt)/b)

def low_estimator(payoff_func, current, nextt, K, r, h):
    b = len(nextt)
    payoff = payoff_func(current, K)
    discount_factor = np.exp(-r*h)
    v_ = np.zeros(b)

    for k in range(b):
        if discount_factor * np.sum(np.concatenate([nextt[:k], nextt[k+1:]]))/(b-1) <= payoff:
            v_[k] = payoff
        else:
            v_[k] = discount_factor * nextt[k]
    return np.sum(v_)/b

In [357]:
def random_tree(X0, m, b, s, r, K, T, payoff_func, estimator):
    w = np.zeros(m + 1).astype(int)
    v = np.zeros((b, m + 1))

    v[0, 0] = X0

    for j in range(1, m + 1):
        v[0, j] = successor(v[0, j-1], s, r, m, T)
        w[j] = 0
    j = m

    while j >= 0:
        if j == m and w[j] < b - 1:
            v[w[j], j] = payoff_func(v[w[j], j], K)
            v[w[j] + 1, j] = successor(v[w[j - 1], j - 1], s, r, m, T)
            w[j] = w[j] + 1
        
        elif j == m and w[j] == b - 1:
            v[w[j], j] = payoff_func(v[w[j], j], K)
            w[j] = 0
            j = j - 1

        elif j < m and w[j] < b - 1:
            v[w[j], j] = estimator(payoff_func, v[w[j], j], v[:, j+1], K, r, T/m) 
            if j > 0:
                v[w[j] + 1, j] = successor(v[w[j - 1], j - 1], s, r, m, T)
                w[j] = w[j] + 1
                for i in range(j + 1, m + 1):
                    v[0, i] = successor(v[w[i - 1], i - 1], s, r, m, T)
                    w[i] = 0
                j = m
            else:
                break

        elif j < m and w[j] == b - 1:
            v[w[j], j] = estimator(payoff_func, v[w[j], j], v[:, j+1], K, r, T/m)
            w[j] = 0
            j = j - 1

    return v[0, 0]

In [358]:
def monte_carlo(n, X0, m, b, s, r, K, T, payoff_func):
    eff_high = []
    for _ in tqdm(range(n)):
        eff_high.append(random_tree(X0, m, b, s, r, K, T, payoff_func, high_estimator))
    return np.array(eff_high)

In [359]:
n = 100
X0 = 100
m = 3
b = 40
s = 0.2
r = 0.1
K = 100
T = 1
eff_high = monte_carlo(n, X0, m, b, s, r, K, T, put_func)

100%|██████████| 100/100 [01:29<00:00,  1.11it/s]


In [361]:
mean_V = np.mean(eff_high)

z = 1.96

std_V = 1/(n-1) * np.sum((eff_high - mean_V)**2)

std_errhigh = z * std_V / np.sqrt(n)
# print('90% confidence interval: [', mean_v - std_errlow, mean_V + std_errhigh,']')

# print('Low estimator:', mean_v)
print('High estimator:', mean_V)
# print('Std error for the low estimator:', std_errlow)
print('Std for the high estimator:', std_errhigh)
print(mean_V + std_errhigh)
# print('Point estimate:', 0.5 * (mean_v + mean_V))

High estimator: 5.814410019098981
Std for the high estimator: 0.13618368052423663
5.950593699623218


In [154]:
def monte_carlo(n, X0, m, b, s, r, K, T, payoff_func):
    eff_low = []
    eff_high = []
    for _ in tqdm(range(n)):
        arbre = rtree(X0, m, b, s, r, K, T, payoff_func)
        eff_low.append(arbre[0])
        eff_high.append(arbre[1])
    return np.array(eff_low), np.array(eff_high)

In [155]:
n = 100
X0 = 1
m = 3
b = 20
s = 0.5
r = 0.1250
K = 1
T = 1
eff_low, eff_high = monte_carlo(n, X0, m, b, s, r, K, T, put_func)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:32<00:00,  3.12it/s]


In [156]:
mean_v = np.mean(eff_low)
mean_V = np.mean(eff_high)

z = 1.96

std_v = 1/(n-1) * np.sum((eff_low - mean_v)**2)
std_V = 1/(n-1) * np.sum((eff_high - mean_V)**2)

std_errlow = z * std_v / np.sqrt(n)
std_errhigh = z * std_V / np.sqrt(n)
print('90% confidence interval: [', mean_v - std_errlow, mean_V + std_errhigh,']')

print('Low estimator:', mean_v)
print('High estimator:', mean_V)
print('Std error for the low estimator:', std_errlow)
print('Std for the high estimator:', std_errhigh)
print('Point estimate:', 0.5 * (mean_v + mean_V))

90% confidence interval: [ 0.11837931998162224 0.1249903383836918 ]
Low estimator: 0.11845689628289215
High estimator: 0.12490616375885708
Std error for the low estimator: 7.757630126990608e-05
Std for the high estimator: 8.417462483473813e-05
Point estimate: 0.12168153002087462


With variance reduction techniques